In [1]:
import os, subprocess, json
from urllib import parse
import sqlalchemy
import nltk
import pandas as pd
import preprocess


with open('./../.API_KEY_/db_key.json', 'r') as f:
    keys = json.load(f)


def log(msg, flag=None):
    if flag==None:
        flag = 0
    head = ["debug", "error", "status"]
    from time import gmtime, strftime
    now = strftime("%H:%M:%S", gmtime())
    if not os.path.isfile("./debug.log"):
        assert subprocess.call(f"echo \"[{now}][{head[flag]}] > {msg}\" > debug.log", shell=True)==0, print(f"[error] > shell command failed to execute")
    else: assert subprocess.call(f"echo \"[{now}][{head[flag]}] > {msg}\" >> debug.log", shell=True)==0, print(f"[error] > shell command failed to execute")


def retrieve_df():
    engine = establish_conn()
    return pd.read_sql_query("select * from english_news_lake", con=engine)


def establish_conn()->sqlalchemy.Engine:
    user = keys['user']
    password = keys['password']
    host = keys['ip']
    port = keys['port']
    database = keys['database']
    password = parse.quote_plus(password)
    engine = sqlalchemy.create_engine(f"mysql://{user}:{password}@{host}:{port}/{database}?charset=utf8mb4")
    return engine


def documents_generator(df:pd.DataFrame):
    log(f"generating documents from dataframe...")
    log(f"iteration init")
    for idx, row in df.iterrows():
        log(f"iteration index : {idx}, row : {row}")
        yield from row['processed_context']


def main():
    log(f"retrieving dataframe from database...")
    df = retrieve_df()
    # 총 문서 갯수
    num_documents = df.shape[0]
    # 각 문서에 대한 라벨 생성
    document_labels = [f"Document{x}" for x in range(1, num_documents+1)]

    # 전처리
    # null 값 처리
    df = preprocess.remove_null_rows(df=df).copy()
    log(f"df shape={df.shape}")
    log(f"null values removed")
    # 전처리작업을 거친 df를 반환
    log(f"processing on rows...")
    df2 = preprocess.preprocess_df(df=df).copy()
    log(f"df2 shape={df2.shape}")
    log("processed columns assigned.")
    # 문서별 정제된 title + context를 기준으로 hash값 생성
    log(f"hash id generating...") 
    df3 = preprocess.create_hash(df2).copy()
    log(f"df3 shape={df3.columns.tolist()}")
    log(f"hash id assigned for each doc.")

In [3]:
# download from nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
"""
변경사항
"""

log(f"retrieving dataframe from database...")
df = retrieve_df()
# 총 문서 갯수
num_documents = df.shape[0]
# 각 문서에 대한 라벨 생성
document_labels = [f"Document{x}" for x in range(1, num_documents+1)]

# 전처리
# null 값 처리
df = preprocess.remove_null_rows(df=df).copy()
log(f"df shape={df.shape}")
log(f"null values removed")
# 전처리작업을 거친 df를 반환
log(f"processing on rows...")
df2 = preprocess.preprocess_df(df=df).copy()
log(f"df2 shape={df2.shape}")
log("processed columns assigned.")
# 문서별 정제된 title + context를 기준으로 hash값 생성
log(f"hash id generating...") 
df3 = preprocess.create_hash(df2).copy()
log(f"df3 shape={df3.columns.tolist()}")
log(f"hash id assigned for each doc.")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Playdata\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Playdata\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Playdata\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
df3.columns.tolist()

['dataSource',
 'title',
 'context',
 'processed_title',
 'processed_context',
 'docKey']

In [9]:
new_columns = ['dataSource', 'title', 'context', 'docKey']

In [10]:
df3 = df3[['dataSource', 'processed_title', 'processed_context', 'docKey']]
df3.columns = new_columns

In [5]:
engine = establish_conn()
log(f"complete set up engine")
df3 = pd.read_sql_query("select * from english_news_warehouse", con=engine)
log(f"success download dataFrame from enligh_news_warehouse dataset")
tokenized_df = preprocess.create_tokens(df3).copy()
log(f"success tokenized")

In [6]:
tokenized_df

,docKey,tokens
0,4178436036041573002,ad sales boost time warner profit quarterly pr...
1,545600611569461556,dollar gains greenspan speech dollar hit highe...
2,9421911741443619047,yukos unit buyer faces loan claim owners embat...
3,231992186597993904,high fuel prices hit ba profits british airway...
4,1188494042733278218,pernod takeover talk lifts domecq shares uk dr...
...,...,...
59507,9415851785762040685,kendall lays pentagon thinking future space pr...
59508,16147701035555579680,larger share noaa declining space budget would...
59509,9119073449501516932,think tank turns attention mars two thousand s...
59510,2660396055900709709,house bill leaves last three jpss satellites l...


In [10]:
tmp_df = pd.DataFrame(df3.loc[:,'title'] + df3.loc[:,'context'])
tmp_df.columns = ['total_text']

In [8]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, BigInteger, DateTime, Date, Text, CHAR, String
from sqlalchemy.dialects.mysql import LONGTEXT
import sqlalchemy
from urllib import parse

engine = establish_conn()

In [8]:
metadata = MetaData()
Table('english_news_warehouse', metadata, 
      Column('dataSource', String(20)),
      Column('title', String(255)),
      Column('context', LONGTEXT),
      Column('docKey', CHAR(20))
      )

metadata.create_all(engine)

In [13]:
metadata = MetaData()
Table('english_news_tokenized', metadata, 
      Column('docKey', CHAR(20)),
      Column('tokens', LONGTEXT)
      )

metadata.create_all(engine)

In [13]:
df3.to_sql("english_news_warehouse",  if_exists='append', con=engine, index=False)

59512

In [14]:
tokenized_df.to_sql("english_news_tokenized",  if_exists='append', con=engine, index=False)

59512